In [35]:
!pip install refinery==0.0.4 --index-url=https://artifactory.prd.it.unity3d.com/artifactory/api/pypi/pypi/simple

Looking in indexes: https://artifactory.prd.it.unity3d.com/artifactory/api/pypi/pypi/simple
  Using cached https://artifactory.prd.it.unity3d.com/artifactory/api/pypi/pypi/unity_vision/0.2.7/unity_vision-0.2.7-py3-none-any.whl (16 kB)
  Attempting uninstall: unity-vision
    Found existing installation: unity-vision 0.2.3
    Not uninstalling unity-vision at /Users/souranilsen/ai/computer-vision/unity-vision, outside environment /Users/souranilsen/ai/computer-vision/unity-vision/venv
    Can't uninstall 'unity-vision'. No files were found to uninstall.
  Attempting uninstall: refinery
    Found existing installation: refinery 0.0.3
    Uninstalling refinery-0.0.3:
      Successfully uninstalled refinery-0.0.3
You should consider upgrading via the '/Users/souranilsen/ai/computer-vision/unity-vision/venv/bin/python -m pip install --upgrade pip' command.


In [36]:
from unity_vision.clients import UCVDClient
from unity_vision.consumers.solo.parser import Solo

In [37]:
client = UCVDClient(
    project_id="3e3b0fb5-50fe-4802-a9eb-db81bae8d185",
    org_id="1374969834147",
    sa_key="a5ba312e-e036-4f79-b0f5-16b18bf8c408",
    api_secret="Nzvy_DRa-clMH0iXZoi4VvCPtd2vrQsw"
)

dest_dir="solo_dir"
dataset_id="803f11ec-cd02-4625-9a67-9b4b0aafca4b"


In [38]:

archives = client.list_dataset_archives(dataset_id)
client.download_dataset_archives(dataset_id=dataset_id, dest_dir="solo_dir", skip_on_error=False)

# TODO: Add API to extract and inflate all archives to one dataset at solo/

In [39]:
solo = Solo(f"{dest_dir}/solo")

DONE (t=0.00025s)


In [ ]:
!refinery local ./pipeline.yml